In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import quicklook_xarray

from gfatpy.atmo.ecmwf import get_ecmwf_temperature_pressure
from gfatpy.atmo.freudenthaler_molecular_properties import molecular_properties
    
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
lidar = preprocess(
    "../tests/datos/MULHACEN/1a/2021/07/05/mhc_1a_Prs_rs_xf_20210705.nc",
    channels = ["532xpa", "532xpp"],
    gluing_products=True
    bin_precision=5
)

In [ ]:
raw = xr.open_dataset("../tests/datos/MULHACEN/1a/2021/07/05/mhc_1a_Prs_rs_xf_20210705.nc")
ranges_between = (75000 < raw.range) & (
        raw.range < 105000
    )

background = raw["signal_532xpa"].loc[:, ranges_between].mean(axis=1)
background_p = raw["signal_532xpp"].loc[:, ranges_between].mean(axis=1)
raw["signal_532xpa"] -= background
raw["signal_532xpp"] -= background_p

In [ ]:
ranges = lidar.range
wavelength = 532
meteo_profiles = get_ecmwf_temperature_pressure(
    "2021-07-05T14:00", heights=ranges
)


# Molecular Attenuated Backscatter
mol_properties = molecular_properties(
    wavelength, meteo_profiles['pressure'], meteo_profiles['temperature'], ranges
)
beta_mol_att = mol_properties["attenuated_molecular_beta"]


In [ ]:
rcs = lidar.signal_532xpa.sel(time=slice("20210705T140000", "20210705T150000")).mean("time") * lidar.range**2
raw_rcs = raw.signal_532xpa.sel(time=slice("20210705T140000", "20210705T150000")).mean("time") * lidar.range**2

nrcs = rcs / rcs.sel(range=slice(6000, 7000)).mean("range")
nraw_rcs = raw_rcs / raw_rcs.sel(range=slice(6000, 7000)).mean("range")
nbeta_mol_att= beta_mol_att / beta_mol_att.sel(range=slice(6000, 7000)).mean("range")


In [ ]:
%matplotlib notebook

nrcs.plot(label = "preprocessed")
nraw_rcs.plot(label = "raw")
nbeta_mol_att.plot(label = "betta mol attenuted")


plt.yscale("log")
plt.ylim(0.01, 10)


plt.legend()

In [ ]:
rcs_a = lidar.signal_532xpa.sel(time=slice("20210705T140000", "20210705T150000")).mean("time") * lidar.range**2
rcs_p = lidar.signal_532xpp.sel(time=slice("20210705T140000", "20210705T150000")).mean("time") * lidar.range**2

nrcsa = rcs_a / rcs_a.sel(range=slice(6000, 7000)).mean("range")
nrcsp = rcs_p / rcs_p.sel(range=slice(6000, 7000)).mean("range")

In [ ]:
%matplotlib notebook

nrcsa.plot(label = "AN bin zero = 5")
nrcsp.plot(label = "PC bin zero = -2")


plt.yscale("log")
plt.ylim(0.01, 10)


plt.legend()

In [ ]:
rcs_a = raw.signal_532xpa.sel(time=slice("20210705T140000", "20210705T150000")).mean("time") * lidar.range**2
rcs_p = raw.signal_532xpp.sel(time=slice("20210705T140000", "20210705T150000")).mean("time") * lidar.range**2

nrcsa = rcs_a / rcs_a.sel(range=slice(2500, 3000)).mean("range")
nrcsp = rcs_p / rcs_p.sel(range=slice(2500, 3000)).mean("range")

In [ ]:
%matplotlib notebook

nrcsa.plot(label = "AN raw")
nrcsp.plot(label = "PC raw")

plt.yscale("log")
plt.ylim(0.01, 10)

plt.grid(c = 'b')


plt.legend()

In [ ]:
quicklook_xarray(lidar.signal_532xpa)
plt.show()